In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
file_dir = "/Users/gaucing/DataAnalytics/movies-etl/Resources"

In [3]:
with open(f"{file_dir}/wikipedia-movies.json", mode="r") as file:
    wiki_movies_raw = json.load(file)

In [4]:
kaggle_metadata = pd.read_csv(f"{file_dir}/movies_metadata.csv", low_memory=False)
ratings = pd.read_csv(f"{file_dir}/ratings.csv")

In [5]:
# remove non-films from data
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]
len(wiki_movies)

7076

In [13]:
# keys for alternate titles
alt_title_keys = ["Also known as","Arabic","Cantonese","Chinese","French",
                  "Hangul","Hebrew","Hepburn","Japanese","Literally",
                  "Mandarin","McCune–Reischauer","Original title","Polish",
                  "Revised Romanization","Romanized","Russian",
                  "Simplified","Traditional","Yiddish"]

# function to clean movie data
def clean_movie(movie):
    movie = dict(movie) # create a non-destructive copy
    
    # remove any alternate title info and re-add it under new "Alternate Title" column
    alt_titles = {}
    for key in alt_title_keys:
        if key in movie:
            alt_titles[key] = movie.pop(key)
    if len(alt_titles) > 0:
        movie["Alternate Titles"] = alt_titles
    
    return movie

In [14]:
# clean movie and create new DataFrame
clean_movies = [clean_movie(movie) for movie in wiki_movies]
clean_movies_df = pd.DataFrame(clean_movies)
sorted(clean_movies_df.columns.tolist())

['Adaptation by',
 'Alternate Titles',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'imdb_link',
 'title',
 'url',
 'year']